In [ ]:
import os
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import numpy as np
from skimage import color
from skimage import io
import pandas as pd
from PIL import Image
import collections
import cv2

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

In [ ]:
def to_index(name):
    return int(name[:6])

In [ ]:
def to_name(index):
    name = ''
    for i in range(6 - len(str(index))):
        name += '0'
    name = name + str(index) + '.jpg'
    return name

In [ ]:
def load_images_from_folder(folder):
    images = {}
    sizes = {}
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder,filename))
        if img is not None: 
            sizes[to_index(filename)] = img.size  
            new_img = img.resize((128,128), Image.ANTIALIAS)
            temp = rgb2gray(np.array(new_img))
            images[to_index(filename)] = temp   
    return images, sizes

In [ ]:
def load_rgb_images_from_folder(folder):
    images = {}
    sizes = {}
    for filename in os.listdir(folder):
        img = Image.open(os.path.join(folder,filename))
        if img is not None: 
            sizes[to_index(filename)] = img.size  
            new_img = img.resize((128,128), Image.ANTIALIAS)
            images[to_index(filename)] = new_img   
    return images, sizes

In [ ]:
def change_bbox(name, old_w, old_h, df):
    df.at[to_index(name)-1, 'x_1'] *= (128/old_w)
    df.at[to_index(name)-1, 'y_1'] *= (128/old_h)
    df.at[to_index(name)-1, 'width'] *= (128/old_w)
    df.at[to_index(name)-1, 'height'] *= (128/old_h)

In [ ]:
folder_name = '../../Data/5000/'
train_imgs, old_sizes= load_images_from_folder(folder_name)

In [ ]:
folder_name = '../../Data/5000/'
train_imgs, old_sizes= load_rgb_images_from_folder(folder_name)

In [ ]:
od = collections.OrderedDict(sorted(train_imgs.items())) #ordered dict

In [ ]:
df = pd.read_csv('../../Data/list_bbox_celeba.csv')

In [ ]:
for i in od.keys():
    img_number = i
    change_bbox(to_name(img_number), old_sizes[img_number][0], old_sizes[img_number][1], df)

In [ ]:
tst = []
for i in range(1,4977):
    tst.append(np.array(od[i]))

In [ ]:
X_train = np.array(tst[:4000])
X_val = np.array(tst[4000:4500])
X_test = np.array(tst[4500:])
Y_train = np.array(df.drop(columns=['image_id'],axis = 1)[:4000])
Y_val = np.array(df.drop(columns=['image_id'],axis = 1)[4000:4500])
Y_test = np.array(df.drop(columns=['image_id'],axis = 1)[4500:5000])

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model = keras.Sequential()
model.add(keras.Input(shape=(128, 128, 3)))
model.add(layers.Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding="valid"))
model.add(layers.PReLU(shared_axes=[1, 2]))
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same"))

model.summary()

model.add(layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding="valid"))
model.add(layers.PReLU(shared_axes=[1, 2]))
model.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="valid"))

model.add(layers.Conv2D(64, kernel_size=(3, 3), strides=(1, 1), padding="valid"))
model.add(layers.PReLU(shared_axes=[1, 2]))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

model.add(layers.GlobalMaxPooling2D())

model.add(layers.Dense(4))
model.compile('adadelta', loss=custom_loss)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
model = Sequential([
        Dense(200, input_dim=X_train.shape[-1]), 
        Activation('relu'), 
        Dropout(0.2), 
        Dense(Y_train.shape[-1])
    ])
model.compile('adadelta', 'mse')

In [ ]:
model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), verbose=2)

In [ ]:
def custom_loss(bbox1, bbox2):
    '''Calculate overlap between two bounding boxes [x, y, w, h] as the area of intersection over the area of unity'''
    x1, y1, w1, h1 = bbox1[0], bbox1[1], bbox1[2], bbox1[3]
    x2, y2, w2, h2 = bbox2[0], bbox2[1], bbox2[2], bbox2[3]

    w_I = min(x1 + w1, x2 + w2) - max(x1, x2)
    h_I = min(y1 + h1, y2 + h2) - max(y1, y2)
    if w_I <= 0 or h_I <= 0:  # no overlap
        return tf.summary.scalar('name', 1, step=None, description=None)

    I = w_I * h_I

    U = w1 * h1 + w2 * h2 - I

    return tf.summary.scalar('name', 1 - I / U, step=None, description=None)


In [ ]:
def IOU(bbox1, bbox2):
    '''Calculate overlap between two bounding boxes [x, y, w, h] as the area of intersection over the area of unity'''
    x1, y1, w1, h1 = bbox1[0], bbox1[1], bbox1[2], bbox1[3]
    x2, y2, w2, h2 = bbox2[0], bbox2[1], bbox2[2], bbox2[3]

    w_I = min(x1 + w1, x2 + w2) - max(x1, x2)
    h_I = min(y1 + h1, y2 + h2) - max(y1, y2)
    if w_I <= 0 or h_I <= 0:  # no overlap
        return 0.
    I = w_I * h_I

    U = w1 * h1 + w2 * h2 - I

    return I / U